In [1]:
import numpy as np
import ebweyl_pub.ebweyl as ebw
import matplotlib.pyplot as plt

# Check Annalytic metric

In [2]:
L = 20
N = 128 
dx = L/N
xyz = np.arange(-L/2, L/2, dx)#.astype('float128')
x, y, z = np.meshgrid(xyz, xyz, xyz, indexing='ij')
t = 1.5
B = (x+np.log(t)) 
Box_0 = np.zeros([N, N, N])
Box_1 = np.ones([N, N, N])
gdown4 = np.array([[-Box_1, Box_0, Box_0, Box_0],
                   [Box_0, (t*t)*Box_1, Box_0, Box_0],
                   [Box_0, Box_0, t*np.exp(x), t*np.exp(x)*B],
                   [Box_0, Box_0, t*np.exp(x)*B, 
                    t*np.exp(x)*(B*B+1)]])
dtB = 1/t
dtgdown4 = np.array([[Box_0, Box_0, Box_0, Box_0],
                     [Box_0, 2*t*Box_1, Box_0, Box_0],
                     [Box_0, Box_0, np.exp(x), 
                      np.exp(x)*B + t*np.exp(x)*dtB],
                     [Box_0, Box_0, np.exp(x)*B + t*np.exp(x)*dtB, 
                      np.exp(x)*(B*B+1) + t*np.exp(x)*(2*dtB*B)]])
Kdown4 = (-1/2)*dtgdown4

In [3]:
FD = ebw.FiniteDifference(dx, N, periodic_boundary=False, fd_order6=True)
EBW = ebw.Weyl(FD, gdown4, Kdown4)

In [4]:
e0, e1, e2, e3 = EBW.tetrad_base(EBW.nup4)
lup, kup, mup, mbup = EBW.null_vector_base(EBW.nup4)

In [5]:
klmdiff = EBW.gup4 - (- ( np.einsum('a...,b...->ab...', lup, kup) + np.einsum('b...,a...->ab...', lup, kup) ) 
                      + ( np.einsum('a...,b...->ab...', mup, mbup) + np.einsum('b...,a...->ab...', mup, mbup) ) )

In [6]:
ediff = EBW.gup4 - (- np.einsum('a...,b...->ab...', e0, e0) + np.einsum('a...,b...->ab...', e1, e1)
                      + np.einsum('a...,b...->ab...', e2, e2) + np.einsum('a...,b...->ab...', e3, e3) )

In [7]:
def print_estimates(v):
    return np.min(v), np.mean(v), np.max(v)

for i in range(4):
    for j in range(4):
        print(i, j, print_estimates(ediff[i,j]))
        
print()
for i in range(4):
    for j in range(4):
        print(i, j, print_estimates(klmdiff[i,j]))

0 0 (0.0, 0.0, 0.0)
0 1 (-0.0, 0.0, -0.0)
0 2 (-0.0, 0.0, -0.0)
0 3 (-0.0, 0.0, -0.0)
1 0 (-0.0, 0.0, -0.0)
1 1 (-8.104628079763643e-15, 1.8735013540549517e-16, 7.93809462606987e-15)
1 2 (-0.0, 0.0, -0.0)
1 3 (-0.0, 0.0, -0.0)
2 0 (-0.0, 0.0, -0.0)
2 1 (-0.0, 0.0, -0.0)
2 2 (-4.190951585769653e-08, -5.451341159586705e-10, 1.0128132998943329e-08)
2 3 (-4.336470738053322e-09, -5.7173955033225425e-11, 1.1059455573558807e-09)
3 0 (-0.0, 0.0, -0.0)
3 1 (-0.0, 0.0, -0.0)
3 2 (-4.336470738053322e-09, -5.7173955033225425e-11, 1.1059455573558807e-09)
3 3 (-4.4929038267582655e-10, -5.99777800383432e-12, 1.2187229003757238e-10)

0 0 ((-2.220446049250313e-16+0j), (-2.220446049250313e-16+0j), (-2.220446049250313e-16+0j))
0 1 ((-0+0j), 0j, (-0+0j))
0 2 ((-0+0j), 0j, (-0+0j))
0 3 ((-0+0j), 0j, (-0+0j))
1 0 ((-0+0j), 0j, (-0+0j))
1 1 ((-7.993605777301127e-15+0j), (2.983724378680108e-16+0j), (8.049116928532385e-15+0j))
1 2 ((-0+0j), 0j, (-0+0j))
1 3 ((-0+0j), 0j, (-0+0j))
2 0 ((-0+0j), 0j, (-0+0j))
2 1

# Check numerical metric

In [8]:
import h5py
from data_analysis_codes import param
from data_analysis_codes.tools import LinData
from data_analysis_codes.tools import GetVars_Plot2d as GVar

In [9]:
N = 64
it = 2500
sim = param.pflrw_d3e2_L1821_t1_N64_LCDM()
Lin = LinData.LinData_Class(sim, '/home/robynm/simulations/'+sim.sim_name+'/')
get_var = GVar.Get_var(Lin, '/home/robynm/simulations/'+sim.sim_name+'/output-0000/pflrw_L1821/')
f = h5py.File('{}_it_{:06d}.hdf5'.format('/home/robynm/simulations/'+sim.sim_name+'/output-0000/pflrw_L1821/all_iterations/'+sim.sim_name, it), 'r')
gdown3 = get_var.get_the_metric(f, it)['gdown']
Kdown3 = get_var.get_the_curv(f, it)['Kdown']

 read time file


In [10]:
Box_zero = np.zeros([N, N, N])
gdown4 = np.array([[Box_zero - 1.0, Box_zero,   Box_zero,   Box_zero],
                   [Box_zero,  gdown3[0,0], gdown3[0,1], gdown3[0,2]],
                   [Box_zero,  gdown3[1,0], gdown3[1,1], gdown3[1,2]],
                   [Box_zero,  gdown3[2,0], gdown3[2,1], gdown3[2,2]]])
Kdown4 = np.array([[Box_zero, Box_zero,   Box_zero,   Box_zero],
                   [Box_zero,  Kdown3[0,0], Kdown3[0,1], Kdown3[0,2]],
                   [Box_zero,  Kdown3[1,0], Kdown3[1,1], Kdown3[1,2]],
                   [Box_zero,  Kdown3[2,0], Kdown3[2,1], Kdown3[2,2]]])

In [11]:
FD = ebw.FiniteDifference(sim.dx, Lin.N, periodic_boundary=True, fd_order6=True)
EBW = ebw.Weyl(FD, gdown4, Kdown4)

In [12]:
e0, e1, e2, e3 = EBW.tetrad_base(EBW.nup4)
ediff = EBW.gup4 - (- np.einsum('a...,b...->ab...', e0, e0) + np.einsum('a...,b...->ab...', e1, e1)
                      + np.einsum('a...,b...->ab...', e2, e2) + np.einsum('a...,b...->ab...', e3, e3) )
lup, kup, mup, mbup = EBW.null_vector_base(EBW.nup4)
klmdiff = EBW.gup4 - (- ( np.einsum('a...,b...->ab...', lup, kup) + np.einsum('b...,a...->ab...', lup, kup) ) 
                      + ( np.einsum('a...,b...->ab...', mup, mbup) + np.einsum('b...,a...->ab...', mup, mbup) ) )

In [13]:
def print_estimates(v):
    return np.min(v), np.mean(v), np.max(v)

for i in range(4):
    for j in range(4):
        print(i, j, print_estimates(ediff[i,j]))
        
print()
for i in range(4):
    for j in range(4):
        print(i, j, print_estimates(klmdiff[i,j]))

0 0 (0.0, 0.0, 0.0)
0 1 (-0.0, 0.0, -0.0)
0 2 (-0.0, 0.0, -0.0)
0 3 (-0.0, 0.0, -0.0)
1 0 (-0.0, 0.0, -0.0)
1 1 (-5.684341886080801e-13, -6.811716989074679e-14, 4.547473508864641e-13)
1 2 (-1.1102230246251565e-16, -1.5675589595474782e-20, 1.1102230246251565e-16)
1 3 (-1.1102230246251565e-16, 2.8145854320005984e-21, 1.3877787807814457e-16)
2 0 (-0.0, 0.0, -0.0)
2 1 (-1.1102230246251565e-16, -1.5675589595474782e-20, 1.1102230246251565e-16)
2 2 (-5.684341886080801e-13, -3.0458491631635276e-14, 4.547473508864641e-13)
2 3 (-1.1102230246251565e-16, -2.4488371841791166e-20, 9.71445146547012e-17)
3 0 (-0.0, 0.0, -0.0)
3 1 (-1.1102230246251565e-16, 2.8145854320005984e-21, 1.3877787807814457e-16)
3 2 (-1.1102230246251565e-16, -2.4488371841791166e-20, 9.71445146547012e-17)
3 3 (-5.684341886080801e-13, -2.396368693757811e-14, 4.547473508864641e-13)

0 0 ((-2.220446049250313e-16+0j), (-2.220446049250313e-16+0j), (-2.220446049250313e-16+0j))
0 1 ((-0+0j), 0j, (-0+0j))
0 2 ((-0+0j), 0j, (-0+0j))
0 3 

# Check analytic case with different e0

In [15]:
n1 = 1.0
n2 = 2.0
n3 = 0.0
n0 = np.sqrt(1 + n1*n1*t*t + n2*n2*np.exp(x)*t)
uup4 = np.array([n0*Box_1, n1*Box_1, n2*Box_1, n3*Box_1])

In [16]:
np.min(et*(1+B*B))

0.006337042491878333

In [17]:
e0, e1, e2, e3 = EBW.tetrad_base(uup4)
lup, kup, mup, mbup = EBW.null_vector_base(uup4)

In [18]:
def print_norm(e):
    en = np.einsum('a...,b...,ab...->...', e, e, gdown4)
    return np.min(en), np.mean(en), np.max(en)
print('e0 norm =', print_norm(e0))
print('e1 norm =', print_norm(e1))
print('e2 norm =', print_norm(e2))
print('e3 norm =', print_norm(e3))

e0 norm = (-1.000000000014552, -1.0000000000000553, -0.9999999999854481)
e1 norm = (0.999999999992724, 1.0000000000001732, 1.000000000014552)
e2 norm = (0.999999999992724, 0.9999999999999667, 1.000000000007276)
e3 norm = (0.9999999999999574, 1.0000000000000002, 1.0000000000000284)


In [19]:
def print_ortho(ea, eb):
    eo = np.einsum('a...,b...,ab...->...', ea, eb, gdown4)
    return np.min(eo), np.mean(eo), np.max(eo)

e = [e0, e1, e2, e3]

for i in range(4):
    for j in range(4):
        if i!=j:
            print('e'+str(i)+' and e'+str(j)+' =', print_ortho(e[i], e[j]))
    print()

e0 and e1 = (-7.275957614183426e-12, 3.9688760006213524e-14, 1.4551915228366852e-11)
e0 and e2 = (-3.637978807091713e-11, -1.304780122921706e-13, 1.8189894035458565e-11)
e0 and e3 = (-1.5266778063960373e-08, 7.087190426538492e-11, 3.083641786361113e-08)

e1 and e0 = (-7.275957614183426e-12, 3.9688760006213524e-14, 1.4551915228366852e-11)
e1 and e2 = (-3.637978807091713e-11, -8.993271351145221e-14, 2.9103830456733704e-11)
e1 and e3 = (-1.2702912499662489e-08, 5.897639158720241e-11, 2.565775503171608e-08)

e2 and e0 = (-3.637978807091713e-11, -1.304780122921706e-13, 1.8189894035458565e-11)
e2 and e1 = (-3.637978807091713e-11, -8.993271351145221e-14, 2.9103830456733704e-11)
e2 and e3 = (-8.987035471363924e-08, -2.4812365400153635e-10, 3.4918230085168034e-08)

e3 and e0 = (-1.5266778063960373e-08, 7.087190426538492e-11, 3.083641786361113e-08)
e3 and e1 = (-1.2702912499662489e-08, 5.897639158720241e-11, 2.565775503171608e-08)
e3 and e2 = (-8.987035471363924e-08, -2.4812365400153635e-10, 3.4

In [20]:
ediff = EBW.gup4 - (- np.einsum('a...,b...->ab...', e0, e0) + np.einsum('a...,b...->ab...', e1, e1)
                      + np.einsum('a...,b...->ab...', e2, e2) + np.einsum('a...,b...->ab...', e3, e3) )
klmdiff = EBW.gup4 - (- ( np.einsum('a...,b...->ab...', lup, kup) + np.einsum('b...,a...->ab...', lup, kup) ) 
                      + ( np.einsum('a...,b...->ab...', mup, mbup) + np.einsum('b...,a...->ab...', mup, mbup) ) )

In [21]:
def print_estimates(v):
    return np.min(v), np.mean(v), np.max(v)

for i in range(4):
    for j in range(4):
        print(i, j, print_estimates(ediff[i,j]))
        
print()
for i in range(4):
    for j in range(4):
        print(i, j, print_estimates(klmdiff[i,j]))

0 0 (-1.4607936553723277e-06, -7.031948783681186e-09, 6.034053654513372e-07)
0 1 (-1.1155173934627409e-13, 8.181697303945284e-16, 2.717321481999069e-13)
0 2 (-1.0131368154190644e-06, -3.6433982671931628e-09, 4.0538812107966013e-07)
0 3 (-4.101995498360409e-08, 3.1888448647579884e-10, 1.0093651881578811e-07)
1 0 (-1.1155173934627409e-13, 8.181697303945284e-16, 2.717321481999069e-13)
1 1 (-1.2935152948756468e-11, -7.729927808952652e-15, 2.5871083053630173e-11)
1 2 (-2.553590577876666e-14, 5.819230871887733e-16, 3.572469646324104e-14)
1 3 (-4.257999378952414e-15, -2.6359430894157334e-17, 3.669871496403379e-15)
2 0 (-1.0131368154190644e-06, -3.6433982671931628e-09, 4.0538812107966013e-07)
2 1 (-2.553590577876666e-14, 5.819230871887733e-16, 3.572469646324104e-14)
2 2 (-1.4021458560192768e-08, 2.2424856576495227e-10, 1.792795956134796e-08)
2 3 (-1.2514647096395493e-09, 2.5904952449108096e-11, 1.862645149230957e-09)
3 0 (-4.101995498360409e-08, 3.1888448647579884e-10, 1.0093651881578811e-07)
